# Demo Notebook how to run models on static mouse datasets

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True

name = "test"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [3]:
import torch
import numpy as np
import pickle 

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder

from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader

Connecting konstantin@sinzlab.chlkmukhxp6i.eu-central-1.rds.amazonaws.com:3306


# Get Dataloader

In [4]:
# change path here
paths = ['data/static22564-2-12-preproc0.zip']

dataset_fn = 'nnsysident.datasets.mouse_loaders.static_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
    seed=1
)
dataloaders = builder.get_data(dataset_fn, dataset_config)

data/static22564-2-12-preproc0 exists already. Not unpacking data/static22564-2-12-preproc0.zip


In [5]:
dataloaders

OrderedDict([('train',
              OrderedDict([('22564-2-12-0',
                            <torch.utils.data.dataloader.DataLoader at 0x7f486f8d5520>)])),
             ('validation',
              OrderedDict([('22564-2-12-0',
                            <torch.utils.data.dataloader.DataLoader at 0x7f486f8d57c0>)])),
             ('test',
              OrderedDict([('22564-2-12-0',
                            <torch.utils.data.dataloader.DataLoader at 0x7f486f8d5730>)]))])

# Get Model

### The old gaussian readout

In [ ]:
model_fn = 'nnvision.models.se_core_gauss_readout'
model_config = {
   'pad_input': False,
   'stack': -1,
   'layers':4,
   'input_kern': 9,
   'gamma_input': 20,
   'gamma_readout': 0.012,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels': 64,
    'depth_separable': True,
    
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1000)

### Spatial Transformer

In [ ]:
model_fn = 'nnvision.models.se_core_spatialXfeature_readout'
model_config = {
   'pad_input': False,
   'stack': -1,
   'layers':4,
   'input_kern': 9,
   'gamma_input': 20,
   'gamma_readout': 0.005,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels': 64,
    'init_noise': 1e-3,
    'depth_separable': True,
    
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1000)

### The New gaussian readout: change gauss_type for the different modes

In [ ]:
model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'
model_config = {
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1000)

In [6]:
model_fn = 'nnsysident.models.models.se2d_spatialxfeaturelinear'
model_config = {
}
model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1000)

In [7]:
model

Encoder(
  (core): SE2dCore(
    (_input_weights_regularizer): LaplaceL2norm(
      (laplace): Laplace()
    )
    (features): Sequential(
      (layer0): Sequential(
        (conv): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (layer1): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (in_depth_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (spatial_conv): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=64, bias=False)
          (out_depth_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (layer2): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (i

# Get Trainer

In [8]:
trainer_fn = 'nnsysident.training.trainers.standard_trainer'
trainer_config = dict()
trainer = builder.get_trainer(trainer_fn, trainer_config)

# Run Training

In [ ]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=123)

Epoch 1: 100%|██████████| 70/70 [00:15<00:00,  4.47it/s]


[001|00/05] ---> 0.11776267737150192


Epoch 2: 100%|██████████| 70/70 [00:09<00:00,  7.44it/s]


[002|00/05] ---> 0.16628608107566833


Epoch 3: 100%|██████████| 70/70 [00:09<00:00,  7.53it/s]


...

Epoch 22: 100%|██████████| 70/70 [00:11<00:00,  6.31it/s]


[022|00/05] ---> 0.26504236459732056


Epoch 23: 100%|██████████| 70/70 [00:12<00:00,  5.43it/s]


[023|01/05] -/-> 0.2628045678138733


Epoch 24: 100%|██████████| 70/70 [00:13<00:00,  5.34it/s]
